In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../Elenco SME_2025-2-ICMC.csv', delimiter=';')

In [3]:
df = df.dropna(subset=["Disciplina (código)"])

In [4]:
df = df.rename(columns={"Disciplina (código)": "codigo", 
                        "Disciplina (nome completo)": "nome", 
                        "Curso(s)": "curso",
                        "Horário 1": "horario1", "Horário 2": "horario2", "Ementa": "ementa", "Turma": "turma"})

In [5]:
df = df.astype({'turma': 'int32'})

In [6]:
df

,codigo,nome,curso,turma,horario1,horario2,nivel,ementa
0,SME0142,Algebra Linear e Aplicações,"BCC, BCDados",1,Terça - 10:10 / 11:50,Quinta - 10:10 / 11:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
1,SME0142,Algebra Linear e Aplicações,BCC,2,Terça - 08:10 / 09:50,Quinta - 08:10 / 09:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
2,SME0123,Estatística,BCC,1,Terça - 14:20 / 16:00,Sexta - 08:10 / 09:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
3,SME0123,Estatística,BCC,2,Terça - 16:20 / 18:00,Sexta - 10:10 / 11:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
4,SME0110,Programação Matemática,BCC,1,Segunda - 08:10 / 09:50,Quarta - 10:10 / 11:50,g,https://uspdigital.usp.br/jupiterweb/obterDisc...
...,...,...,...,...,...,...,...,...
58,SME5962,Otimização Inteira,Otimização,1,a definir,NaN,p,pos
59,SMA5934,Tópicos em Teoria Algébrica de Invariantes e B...,Matemática,1,a definir,NaN,p,pos
60,SMA5802,Equações Diferenciais Ordinárias,Matemática,1,a definir,NaN,p,pos
61,SME5873,Ferramentas de Programação Paralela para Soluç...,MFC,1,a definir,NaN,p,pos


In [7]:
json_data = df.to_dict(orient='records')

In [8]:
for item in json_data:
    keys_to_remove = [k for k, v in item.items() if pd.isna(v) and k == "horario2"]
    for k in keys_to_remove:
        del item[k]

In [9]:
len(json_data)

63

In [10]:
dias_map = {
    "Segunda": "Seg.",
    "Terça": "Ter.",
    "Quarta": "Qua.",
    "Quinta": "Qui.",
    "Sexta": "Sex.",
    "Sábado": "Sáb.",
    "Domingo": "Dom."
}

In [11]:
def processar_horario(horario):
    if pd.isna(horario):
        return None
    try:
        # Exemplo: "Terça - 14:20 / 16:00"
        dia_e_horas = horario.split('-')
        dia_extenso = dia_e_horas[0].strip()
        dia = dias_map.get(dia_extenso, dia_extenso)  # Se não tiver no mapa, mantém original
        horas = dia_e_horas[1].strip().split('/')
        inicio = horas[0].strip()
        fim = horas[1].strip()
        return {"dia": dia, "inicio": inicio, "fim": fim}
    except Exception as e:
        print(f"Erro ao processar: {horario} -> {e}")
        return None

In [12]:
for item in json_data:
    horarios = []
    for key in ['horario1', 'horario2']:
        if key in item:
            h = processar_horario(item[key])
            if h:
                horarios.append(h)
            del item[key]  # Remove os campos antigos

    # if horarios:
    item['horarios'] = horarios
    item['id'] = f'{item['codigo']},{item['turma']}'


Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: 
Sex. 10:10/11:50 -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar: a def

In [13]:
len(json_data)

63

In [14]:
import json

In [15]:
with open('turmas.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)